In [1]:
import sqlite3

In [21]:
con = sqlite3.connect('development.db')
cur = con.cursor()

In [22]:
# Creating Tables

cur.execute('''CREATE TABLE IF NOT EXISTS transactions
               (date text, 
               type text, 
               symbol text, 
               qty real, 
               price real)''')

cur.execute('''CREATE TABLE IF NOT EXISTS balance
               (symbol text, 
               qty real)''')

In [23]:
def updateBalance():
    _,Type,Symbol,Qty,_ = cur.execute('SELECT * FROM transactions ORDER BY rowid DESC LIMIT 1').fetchone()
    
    DBSearch = cur.execute("SELECT * FROM balance WHERE symbol = '{}'".format(Symbol)).fetchone()
     
    if DBSearch:
        _, Balance = DBSearch
        
        def __update(Value, Symbol):
            cur.execute("UPDATE balance SET qty = {} WHERE symbol = '{}'".format(Value, Symbol))
        
        if Type == 'buy':
            __update(Balance + Qty,Symbol)
        else:
            __update(Balance - Qty,Symbol)
    
    else:
        def __insert(Value, Symbol):
            cur.execute("INSERT INTO balance (symbol, qty) VALUES ('{}',{})".format(Symbol, Value))
        
        if Type == 'buy':
            __insert(Qty, Symbol)
        else:
            __insert(-Qty, Symbol)
        
    cur.execute('DELETE FROM balance WHERE qty = 0')
    

def registerOrder(Type,Symbol,Qty,Price):
    # Insert a row of data
    cur.execute('''INSERT INTO transactions
        (date, type, symbol, qty, price)
        VALUES
        (datetime('now', 'localtime'),'{}','{}',{},{})'''.format(Type, Symbol, Qty, Price))
    
    updateBalance()
    
    con.commit()

In [8]:
registerOrder('sell','AAPL',100,72.80)

In [27]:
registerOrder('buy','AAPL',100,72.80)

In [10]:
registerOrder('sell','PETR4',100,72.80)

In [26]:
registerOrder('buy','PETR4',100,72.80)

In [28]:
for row in cur.execute('SELECT * FROM balance ORDER BY rowid DESC'):
        print(row)

('AAPL', 200.0)
('PETR4', 200.0)


In [29]:
for row in cur.execute('SELECT rowid,* FROM transactions ORDER BY rowid DESC'):
        print(row)

(8, '2022-03-02 19:30:00', 'buy', 'AAPL', 100.0, 72.8)
(7, '2022-03-02 19:29:57', 'buy', 'PETR4', 100.0, 72.8)
(6, '2022-03-02 19:25:01', 'buy', 'AAPL', 100.0, 72.8)
(5, '2022-03-02 19:24:57', 'buy', 'PETR4', 100.0, 72.8)
(4, '2022-03-02 19:24:24', 'buy', 'PETR4', 100.0, 72.8)
(3, '2022-03-02 19:24:23', 'sell', 'PETR4', 100.0, 72.8)
(2, '2022-03-02 19:24:22', 'buy', 'AAPL', 100.0, 72.8)
(1, '2022-03-02 19:24:22', 'sell', 'AAPL', 100.0, 72.8)


In [19]:
# Close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()